## Example SpikeForest import

This notebook will import a few SpikeForest recordings into the labbox_ephys database

In [ ]:
# Define some functions

import lbdb
import kachery as ka
import labbox_ephys as le
import hither
import numpy as np

def geom_from_recording(recording):
    channel_ids = recording.get_channel_ids()
    location0 = recording.get_channel_property(channel_ids[0], 'location')
    nd = len(location0)
    M = len(channel_ids)
    geom = np.zeros((M, nd))
    for ii in range(len(channel_ids)):
        location_ii = recording.get_channel_property(channel_ids[ii], 'location')
        geom[ii, :] = list(location_ii)
    return geom

def import_recording(*, recording_id, recording):
    print(f'Importing {recording_id}')
    doc = dict(
        recording_id=recording_id,
        recording=recording.object(),
        sampling_frequency=recording.get_sampling_frequency(),
        channel_ids=recording.get_channel_ids(),
        geom=geom_from_recording(recording).tolist(),
        num_frames=recording.get_num_frames()
    )
    le.dbcollection('recordings').replace_one(dict(recording_id=recording_id), doc, upsert=True)
    
def store_sorting_in_database(*, recording, sorting, recording_id, group_id, sorting_name, sorter_name, sorting_params, runtime_info):
    sorting_id = f'{recording_id}::{group_id}::{sorting_name}'
    doc = dict(
        recording_id=recording_id,
        group_id=group_id,
        recording=recording.object(),
        sorting=sorting.object(),
        sorting_name=sorting_name,
        sorter_name=sorter_name,
        sorting_id=sorting_id,
        unit_ids=sorting.get_unit_ids().tolist(),
        runtime_info=runtime_info
    )
    le.dbcollection('sortings').replace_one(dict(sorting_id=sorting_id), doc, upsert=True)

def import_recording_with_truth(recording_id, recording, sorting_true):
    import_recording(recording_id=recording_id, recording=recording)
    store_sorting_in_database(recording_id=recording_id, recording=recording, sorting_name='truth', sorting=sorting_true, group_id=0, sorter_name='truth', sorting_params=dict(), runtime_info=None)

In [ ]:
# Import some example SpikeForest recordings

with ka.config(fr='default_readonly'):
    # SYNTH_MAGLAND example
    import_recording_with_truth(
        recording_id='sf:synth_magland_noise10_K10_C4/001_synth',
        recording=le.LabboxEphysRecordingExtractor('sha1://ee5214337b2e01910a92c3613a4b8ad4be4dc476/SYNTH_MAGLAND/synth_magland_noise10_K10_C4/001_synth.json'),
        sorting_true=le.LabboxEphysSortingExtractor('sha1://ff64f1713227c017052bf21b41ddf764320aa606/SYNTH_MAGLAND/synth_magland_noise10_K10_C4/001_synth.firings_true.json')
    )
    
    # PAIRED_KAMPFF example
    import_recording_with_truth(
        recording_id='sf:paired_kampff/2014_11_25_Pair_3_0',
        recording=le.LabboxEphysRecordingExtractor('sha1://b96e34b72c258be205e1bf2a7aaaad72dfd8e351/PAIRED_KAMPFF/paired_kampff/2014_11_25_Pair_3_0.json'),
        sorting_true=le.LabboxEphysSortingExtractor('sha1://86f472cba745921dabb9b4e745f2e3547f9fb3e9/PAIRED_KAMPFF/paired_kampff/2014_11_25_Pair_3_0.firings_true.json')
    )
    
    # PAIRED_BOYDEN example
    import_recording_with_truth(
        recording_id='sf:paired_boyden32c/1103_1_1',
        recording=le.LabboxEphysRecordingExtractor('sha1://0c89b66696ff92e529d97a31fe41af7949111a9a/PAIRED_BOYDEN/paired_boyden32c/1103_1_1.json'),
        sorting_true=le.LabboxEphysSortingExtractor('sha1://fe3f7c918200ffe44db7759911e80978540d5201/PAIRED_BOYDEN/paired_boyden32c/1103_1_1.firings_true.json')
    )

In [ ]:
# List all of the recordings in the database

recordings = le.dbcollection('recordings').find(dict())
for rec in recordings:
    print(rec['recording_id'])

In [ ]:
# List all of the recordings in the database

recordings = le.dbcollection('sortings').find(dict())
for rec in recordings:
    print(rec['sorting_id'])